# Segmented Mirror for HCIPy

I need a segmented mirror for phase control.

In [ ]:
from hcipy import*
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir('../../pastis/')
from config import CONFIG_PASTIS

<span style="color:red"> *** Careful: *** </span>

**The segmented mirror in this notebook is based specifically on the classes and funcitons in the custom module `atlast_imaging`, which is deprecated. I have switched to using the `SegmentedDeformableMirror` from hcipy, however if you run this notebook as is, it should still work fine. Please see the up-to-date notebooks in the main directories for the current versions.**

**You should also be on the PASTIS commit `caf29ad`.**

In [ ]:
import atlast_imaging as atim

In [ ]:
# This is the function I use in PASTIS for making a HiCAT/ATLAST aperture
""" This was the first development model, please use the one from the module atlast_imaging now

def get_atlast_aperture(normalized=False, with_segment_gaps=True, segment_transmissions=1):
    '''Make the ATLAST pupil mask.

    This function is a copy of make_hicat_aperture(), except that it also returns the segment positions.

    Parameters
    ----------
    normalized : boolean
        If this is True, the outer diameter will be scaled to 1. Otherwise, the
        diameter of the pupil will be 15.0 meters.
    with_segment_gaps : boolean
        Include the gaps between individual segments in the aperture.
    segment_transmissions : scalar or array_like
        The transmission for each of the segments. If this is a scalar, this transmission will
        be used for all segments.

    Returns
    -------
    Field generator
        The ATLAST aperture.
    CartesianGrid
        The segment positions.
    '''
    pupil_diameter = 15. # m
    segment_circum_diameter = 2 / np.sqrt(3) * pupil_diameter / 7
    num_rings = 3
    segment_gap = 1e-2

    if not with_segment_gaps:
        segment_gap = 0

    if normalized:
        segment_circum_diameter /= pupil_diameter
        segment_gap /= pupil_diameter
        pupil_diameter = 1.0

    segment_positions = make_hexagonal_grid(segment_circum_diameter / 2 * np.sqrt(3), num_rings)
    segment_positions = segment_positions.subset(lambda grid: ~(circular_aperture(segment_circum_diameter)(grid) > 0))

    hexagon = hexagonal_aperture(segment_circum_diameter - segment_gap)
    
    def segment(grid):
        return hexagon(grid.rotated(np.pi/2))

    segmented_aperture = make_segmented_aperture(segment, segment_positions, segment_transmissions)

    def func(grid):
        res = segmented_aperture(grid)

        return Field(res, grid)
    return func, segment_positions
"""

In [ ]:
''' This was the first development model, please use the one from the module atlast_imaging now

class SegmentedMirror(OpticalElement):
    """A segmented mirror from a segmented aperture.

    Parameters:
    ----------
    aperture : Field
        The segmented aperture of the mirror.
    seg_pos : CartesianGrid(UnstructuredCoords)
        Segment positions of the aperture.
    """

    def __init__(self, aperture, seg_pos):
        self.aperture = aperture
        self.segnum = len(seg_pos.x)
        self.segmentlist = np.arange(1, self.segnum + 1)
        self._coef = np.zeros((self.segnum, 3))
        self.seg_pos = seg_pos
        self.input_grid = aperture.grid
        self._last_npix = np.nan    # see _setup_arrays for this

    def forward(self, wavefront):
        """Propagate a wavefront through the segmented mirror.

        Parameters
        ----------
        wavefront : Wavefront
            The incoming wavefront.

        Returns
        -------
        Wavefront
            The reflected wavefront.
        """
        wf = wavefront.copy()
        wf.electric_field *= np.exp(2j * self.surface * wavefront.wavenumber)
        return wf

    def backward(self, wavefront):
        """Propagate a wavefront backwards through the deformable mirror.

        Parameters
        ----------
        wavefront : Wavefront
            The incoming wavefront.

        Returns
        -------
        Wavefront
            The reflected wavefront.
        """
        wf = wavefront.copy()
        wf.electric_field *= np.exp(-2j * self.surface * wavefront.wavenumber)
        return wf

    @property
    def surface(self):
        """ The surface of the segmented mirror in meters.
        """
        surf = self.apply_coef()
        return surf

    @property
    def coef(self):
        """ The surface shape of the deformable mirror, in **meters**.
        """
        return self._coef

    def flatten(self):
        """ Flatten the DM by setting all segment coefficients to zero."""
        self._coef[:] = 0

    def set_segment(self, segid, piston, tip, tilt):
        """ Set an individual segment of the DM.
        Parameters
        -------------
        segid : integer
            Index of the actuator you wish to control, starting at 1 (center is 0)
        piston, tip, tilt : floats, meters and radians
            Piston (in meters) and tip and tilt (in radians)
        """
        self._coef[segid - 1] = [piston, tip, tilt]

    def _setup_arrays(self):
        """ Set up the arrays to compute an OPD into.
        This is relatively slow, but we only need to do this once for
        each size of input array.
        """
        npix = self.aperture.shaped.shape[0]
        if npix == self._last_npix:
            return
        else:
            self._last_npix = npix

        x, y = self.input_grid.coords

        self._seg_mask = np.zeros_like(x)
        self._seg_x = np.zeros_like(x)
        self._seg_y = np.zeros_like(y)
        self._seg_indices = dict()

        pupil_grid = make_pupil_grid(dims=npix, diameter=1)
        aper_num, seg_positions = get_atlast_aperture(normalized=True,
                                                      segment_transmissions=np.arange(1, self.segnum + 1))
        aper_num = evaluate_supersampled(aper_num, pupil_grid, 2)

        self._seg_mask = np.copy(aper_num)

        for i in self.segmentlist:
            wseg = np.where(self._seg_mask == i)
            self._seg_indices[i] = wseg

            cenx, ceny = self.seg_pos.points[i - 1]

            self._seg_x[wseg] = x[wseg] - cenx
            self._seg_y[wseg] = y[wseg] - ceny
            
            # Set gaps to zero
            bad_gaps_x = np.where(np.abs(self._seg_x) > 0.1)
            self._seg_x[bad_gaps_x] = 0
            bad_gaps_y = np.where(np.abs(self._seg_y) > 0.1)
            self._seg_y[bad_gaps_y] = 0

    def apply_coef(self):
        """ Apply the DM shape from its own segment coefficients
        """
        self._setup_arrays()

        self.opd = np.zeros_like(self._seg_x)
        for i in self.segmentlist:
            wseg = self._seg_indices[i]
            self.opd[wseg] = (self._coef[i - 1, 0] +
                              self._coef[i - 1, 1] * self._seg_x[wseg] +
                              self._coef[i - 1, 2] * self._seg_y[wseg])
        return Field(self.opd, self.input_grid)

    def phase_for(self, wavelength):
        """Get the phase that is added to a wavefront with a specified wavelength.

        Parameters
        ----------
        wavelength : scalar
            The wavelength at which to calculate the phase deformation.

        Returns
        -------
        Field
            The calculated phase deformation.
        """
        return 2 * self.surface * 2 * np.pi / wavelength
'''

In [ ]:
# Parameters
which_tel = CONFIG_PASTIS.get('telescope', 'name')
pupil_size = CONFIG_PASTIS.getint('numerical', 'tel_size_px')
PUP_DIAMETER = CONFIG_PASTIS.getfloat(which_tel, 'diameter')

npix = pupil_size
wvln_hc = 638e-9
lamD = 15
samp = 4
diam = PUP_DIAMETER
norm = False

In [ ]:
pupil_grid = make_pupil_grid(dims=npix, diameter=PUP_DIAMETER)
focal_grid = make_focal_grid(pupil_grid, 8, 20, wavelength=wvln_hc)
prop = FraunhoferPropagator(pupil_grid, focal_grid)

In [ ]:
aper, seg_pos = atim.get_atlast_aperture(normalized=norm)
aper = evaluate_supersampled(aper, pupil_grid, 2)

imshow_field(aper)

## Testing parts of the `SegmentedMirror()` class.

### Init equivalent

In [ ]:
# Init equivalent
aperture = aper
segnum = 36
segmentlist = np.arange(1, segnum+1)
coef = np.zeros((segnum, 3))
seg_pos = seg_pos
input_grid = aperture.grid

### `setup_arrays`

In [ ]:
# setup_arrays
npix = aperture.shaped.shape[0]
x, y = input_grid.coords

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(np.reshape(x, (npix, npix)))
plt.title('x')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(np.reshape(y, (npix, npix)))
plt.title('y')
plt.colorbar()

In [ ]:
_seg_mask = np.zeros_like(x)
_seg_x = np.zeros_like(x)
_seg_y = np.zeros_like(y)
_seg_indices = dict()

plt.subplot(1, 2, 1)
plt.imshow(np.reshape(_seg_x, (npix, npix)))
plt.title('x')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(np.reshape(_seg_y, (npix, npix)))
plt.title('y')
plt.colorbar()

In [ ]:
aper_num, seg_positions = atim.get_atlast_aperture(normalized=norm, segment_transmissions=np.arange(1,segnum+1))
aper_num = evaluate_supersampled(aper_num, pupil_grid, 2)

In [ ]:
imshow_field(aper_num)

In [ ]:
# You can cycle through the segments here
segseg = 6
imshow_field(aper_num, vmin=segseg-0.1, vmax=segseg+0.1)

In [ ]:
_seg_mask = np.copy(aper_num)
plt.imshow(np.reshape(_seg_mask, (npix, npix)))

In [ ]:
segseg = 7
plt.imshow(np.reshape(_seg_mask, (npix, npix)), vmin=segseg-0.1, vmax=segseg+0.1)

In [ ]:
for i in segmentlist:
    #print(i)
    
    wseg = np.where(_seg_mask == i)
    _seg_indices[i] = wseg

    cenx, ceny = seg_pos.points[i-1]
    
    _seg_x[wseg] = x[wseg] - cenx
    _seg_y[wseg] = y[wseg] - ceny

# Set gaps to zero
bad_gaps_x = np.where(np.abs(_seg_x) > 0.1*diam)
_seg_x[bad_gaps_x] = 0
bad_gaps_y = np.where(np.abs(_seg_y) > 0.1*diam)
_seg_y[bad_gaps_y] = 0

In [ ]:
plt.figure(figsize=(18, 9))
plt.subplot(1, 2, 1)
plt.imshow(np.reshape(_seg_x, (npix, npix)))#, vmin=-0.1, vmax=0.1)
plt.title('x')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(np.reshape(_seg_y, (npix, npix)))#, vmin=-0.1, vmax=0.1)
plt.title('y')
plt.colorbar()

In [ ]:
seg_pos.points.shape

### `apply_coef`

In [ ]:
# apply_coef 
opd = np.zeros_like(_seg_x)
for i in segmentlist:
    wseg = _seg_indices[i]
    opd[wseg] = (coef[i-1, 0] +
                        coef[i-1, 1] * _seg_x[wseg] +
                        coef[i-1, 2] * _seg_y[wseg])
res = Field(opd, input_grid)

In [ ]:
# This should be zero
imshow_field(res)

In [ ]:
coef[5] = (4, 5, 6)

In [ ]:
opd = np.zeros_like(_seg_x)
for i in segmentlist:
    wseg = _seg_indices[i]
    opd[wseg] = (coef[i-1, 0] +
                        coef[i-1, 1] * _seg_x[wseg] +
                        coef[i-1, 2] * _seg_y[wseg])
res = Field(opd, input_grid)
imshow_field(res)

## Testing the `SegmentedMirror()` class itself

### Instantiate a segmented mirror

In [ ]:
# Instantiate the segmented mirror
sm = atim.SegmentedMirror(aper, seg_pos)

In [ ]:
# Print the x positions of the segments
print(sm.seg_pos.x.shape)
print(sm.seg_pos.x)

In [ ]:
# Print the y-positions of the segments
print(sm.seg_pos.y.shape)
print(sm.seg_pos.y)

### Make a wavefront that will propagte through the segmented mirror optical element

In [ ]:
# Generate a wavefront on the aperture, which will be a field
# at a specified wavelength (default 1) and display its
# intensity and phase.
wf = Wavefront(aper, wavelength=wvln_hc)

plt.figure(figsize=(18, 9))
plt.suptitle('Wavefront')

plt.subplot(1, 2, 1)
imshow_field(wf.intensity, mask=aper)
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(wf.phase, mask=aper, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

In [ ]:
# Show segment coefficients
print(sm.coef.shape)
print(sm.coef)

### Propagate the WF "through" SM and show at pupil

In [ ]:
# Propagate the wf through the SM and display its intensity
# and phase in that pupil plane, before propagation to image plane.
wf_at_sm = sm(wf)

plt.figure(figsize=(18, 9))
plt.suptitle('WF prop through SM, still in pupil plane')

plt.subplot(1, 2, 1)
imshow_field(wf_at_sm.intensity, mask=aper)
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(wf_at_sm.phase, mask=aper, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

### Propagate from SM pupil to image

In [ ]:
# Propagate from SM to image plane and display intensity and phase
im = prop(sm(wf))

plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after SM')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im.intensity))
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(im.phase, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

### Set segment coefficients for new OPD

In [ ]:
# Set some of the segment coefficients

# either by directly talking to self.coef
#sm.coef[0] = (0.001, 0, 0)
print(sm.coef)

In [ ]:
# or by using set_segment
aber_m = 50e-9
#sm.set_segment(36, aber_m, 0, 0)
#sm.set_segment(35, aber_m, 000, 0d0)
for i in [2, 34, 3, 19, 23,]:
    sm.set_segment(i, 0, aber_m, aber_m)
print(sm.coef)

#### Get the WF at pupil

In [ ]:
# Get the OPD with those coeffs
wf_at_sm2 = sm(wf)

plt.figure(figsize=(18, 9))
plt.suptitle('WF prop through SM, still in pupil plane')

plt.subplot(1, 2, 1)
imshow_field(wf_at_sm2.intensity, mask=aper)
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(wf_at_sm2.phase, mask=aper, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

#### Propagate the WF to image plane

In [ ]:
# Show the image of that
im = prop(sm(wf))

plt.figure(figsize=(18, 9))
plt.suptitle('Image plane after SM')

plt.subplot(1, 2, 1)
imshow_field(np.log10(im.intensity))
plt.title('Intensity')
plt.colorbar()

plt.subplot(1, 2, 2)
imshow_field(im.phase, cmap='RdBu')
plt.title('Phase')
plt.colorbar()

#### Flatten the SM

In [ ]:
sm.flatten()